In [ ]:
import sys,os
# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

sys.path.append(ROOT_DIR)

In [ ]:
import torch
from torchvision import models
import torch.nn as nn
import numpy as np
import pandas as pd
import cv2
from PIL import Image
from dataset.dataset_ImagePrivacy import full_transform
from networks.channel_grouping import load_cls_model
from sklearn.cluster import KMeans

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
print(os.getpid())

## Load the model and hook the features

In [ ]:
features_blobs = []
def hook_feature(module, input, output):
    features_blobs.append(np.squeeze(output.data.cpu().numpy()))

In [ ]:
def load_model():
    model_path = '../../models/ResNet4IP.pth'
    
    model = load_cls_model(class_num=2, pretrained=False)
    model = model.to(device)
    
    model.load_state_dict(torch.load(model_path))
    model.eval()
    
    features_names = ['layer4','avgpool'] # the last conv layer of the resnet101
    for name in features_names:
        model._modules.get(name).register_forward_hook(hook_feature)
        
    return model

In [ ]:
cls_model = load_model()

## Load Data

In [ ]:
partition = 1
partition = str(partition)

In [ ]:
data_dir = '../../../../data/image_privacy/exp/'
data_file = data_dir + 'partition'+ partition + '/train.csv'

data_list = pd.read_csv(data_file)
imgs_name = data_list['img_name'].values.tolist()

## Visulization of CAM

In [ ]:
import random
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [ ]:
# get the softmax weight
params = list(cls_model.parameters())
weight_softmax = params[-2].data.numpy()
weight_softmax[weight_softmax<0] = 0

In [ ]:
def returnCAM(feature_conv, weight_softmax, class_idx):
    # generate the class activation maps upsample to 256x256
    size_upsample = (256, 256)
    nc, h, w = feature_conv.shape
    output_cam = []
    for idx in class_idx:
        cam = weight_softmax[class_idx].dot(feature_conv.reshape((nc, h*w)))
        cam = cam.reshape(h, w)
        cam = cam - np.min(cam)
        cam_img = cam / np.max(cam)
        cam_img = np.uint8(255 * cam_img)
        output_cam.append(cv2.resize(cam_img, size_upsample))
    return output_cam

In [ ]:
# img_name = random.choice(imgs)
# img_name = img_dir + img_name

img_name = random.choice(imgs_name)

img = Image.open(img_name)
input_img = full_transform(img).unsqueeze(0)

plt.imshow(img)
plt.show()

In [ ]:
# forward pass, the result is the probability of [private, public]
features_blobs = []
logit = cls_model.forward(input_img)
h_x = F.softmax(logit, 1).data.squeeze()
probs, idx = h_x.sort(0, True)
probs = probs.numpy()
idx = idx.numpy()
print(probs)

In [ ]:
print('RESULT ON ' + img_name)

# generate class activation mapping
print('Class activation map is saved as cam.jpg')
CAMs = returnCAM(features_blobs[0], weight_softmax, [idx[0]])

# render the CAM and output
img = cv2.imread(img_name)
height, width, _ = img.shape
heatmap = cv2.applyColorMap(cv2.resize(CAMs[0],(width, height)), cv2.COLORMAP_JET)
result = heatmap * 0.4 + img * 0.5
# cv2.imwrite('cam.jpg', result)

b,g,r=cv2.split(result)
img_rgb = cv2.merge([r,g,b])

plt.imshow(img_rgb/255)
plt.show()

## get coordinate of peak response  (features for each channel)

In [ ]:
channel_features = []

for i in range(2048): # corresponding to the channels
    channel_features.append([])

In [ ]:
for img_name in imgs_name:
    try:
        img = Image.open(img_name).convert('RGB') # convert gray to rgb
        input_img = full_transform(img).unsqueeze(0).to(device)

    #     plt.imshow(img)
    #     plt.show()

        features_blobs = []
        logit = cls_model.forward(input_img)

        for i, channel in enumerate(features_blobs[0]):

            tx, ty =  np.where(channel==channel.max())
            tx, ty = tx[0], ty[0]

            channel_features[i].append(tx)
            channel_features[i].append(ty)
    except Exception as e:
        print(e)
        print(img_name)

channel_features = np.array(channel_features)
np.save('./channel_features.npy', channel_features)


In [ ]:
print(channel_features.shape) # channel_feature: [2048, 2 * sizeof_train_images]

## channel clustering

In [ ]:
channel_features = np.load('./channel_features.npy')

In [ ]:
part_nums = list(range(2,14,2))

In [ ]:
part_nums

In [ ]:
for part_num in part_nums:
    part_index = KMeans(n_clusters=part_num, random_state=9).fit_predict(channel_features)
    np.save('./grouping_result/channel_cluster_' + str(part_num) + '.npy', part_index)